# TO DO: CARGAR LOS OUTPUTS DE LOS DOS MODELOS

In [1]:
import pandas as pd

lstm_probs_train = pd.read_csv('/kaggle/input/lstm-results/predicts_vs_real_TRAIN.csv', index_col=0)
lstm_probs_test = pd.read_csv('/kaggle/input/lstm-results/predicts_vs_real_TEST.csv', index_col=0)
lstm_columns = ['lstm_prob_0', 'lstm_prob_1', 'lstm_prob_2', 'lstm_prob_3', 'lstm_prob_4', 'lstm_prob_5', 'tag']

In [2]:
tree_probs_train = pd.read_csv('/kaggle/input/tree-results/predicts_vs_real_TRAIN.csv', index_col=0)
print('Train checker:', all(tree_probs_train.tag.values == lstm_probs_train.tag.values))
tree_probs_train.drop(columns = 'tag', inplace=True)

tree_probs_test = pd.read_csv('/kaggle/input/tree-results/predicts_vs_real_TEST.csv', index_col=0)
print('Test checker:', all(tree_probs_test.tag.values == lstm_probs_test.tag.values))
tree_probs_test.drop(columns = 'tag', inplace=True)
tree_columns = ['tree_prob_0', 'tree_prob_1', 'tree_prob_2', 'tree_prob_3', 'tree_prob_4', 'tree_prob_5']

Train checker: True
Test checker: True


# TO DO: CREAR DOS CONJUNTOS TRAIN Y TEST CON SUS X E Y

In [3]:
lstm_probs_train.columns = lstm_probs_test.columns = lstm_columns
tree_probs_train.columns = tree_probs_test.columns = tree_columns

x_train = pd.concat([tree_probs_train, lstm_probs_train], axis=1)
y_train = x_train.tag
x_train.drop(columns='tag', inplace=True)

x_test = pd.concat([tree_probs_test, lstm_probs_test], axis=1)
y_test = x_test.tag
x_test.drop(columns='tag', inplace=True)

# TO DO: ENTRENAR UN MODELO DE REGRESION QUE HAGA DE ENSEMBLE ENTRE LOS DOS DE PARTIDA

In [4]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

lr_model = LogisticRegression(max_iter=5000)

parameters = {}
parameters['C'] = [10e-3, 10e-2, 10e-1, 1, 10, 100, 1000]
parameters['class_weight'] = [None, 'balanced']

GS = GridSearchCV(lr_model, parameters)
GS.fit(x_train, y_train)

lr_model = GS.best_estimator_

pred_probs_test = lr_model.predict_proba(x_test)
y_test_pred = np.argmax(pred_probs_test, axis=1)
report = classification_report(y_test, y_test_pred, output_dict=True)
pd.DataFrame(report).T

,precision,recall,f1-score,support
0,0.639289,0.801033,0.711079,14138.000000
1,0.802124,0.244844,0.375169,3394.000000
2,0.348275,0.308694,0.327292,11905.000000
3,0.727968,0.716033,0.721951,7441.000000
4,0.538642,0.586073,0.561357,10584.000000
5,0.629449,0.596672,0.612623,12982.000000
accuracy,0.580835,0.580835,0.580835,0.580835
macro avg,0.614291,0.542225,0.551579,60444.000000
weighted avg,0.582294,0.580835,0.570602,60444.000000


Mejoramos **~1%** con respecto a la media aritmetica de las predicciones